pair_multiplication is a Python library for working with Young Diagrams and pairs of Young diagrams.


## Features

- **NullDiagram**: Handles null cases of Young diagrams.
- **YoungDiagram** inherits from *NullDiagram*: A class to create and manipulate Young diagrams.
- **Pair** inherits from *YoungDiagram*: Represents a pair of Young diagrams.
- **DirectSum** inherits from *dict*: Creates a direct sum of diagrams.
- **DimensionDirectSum** inherits from *DirectSum*: Useful for quickly showing the dimensions and multiplicities of a direct sum

## Installation

Install the package using `pip`:

```bash
pip install git+https://github.com/BernieTelalovic/pair_multiplication.git

```

In [1]:
from pair_multiplication import *

# YoungDiagram class construction

Starting from a partition tuple labelling a diagram, $(a_1,a_2,..)$ with $a_1\geq a_2\geq...$ being the number of boxes in each row labelled by the subscript, we can construct young diagrams:

In [2]:
yd = YoungDiagram((3,2,1))

but not:

In [3]:
yd = YoungDiagram((1,2,3)) # this is supposed to give an error, don't worry

ValueError: Not a young diagram.

### Quark-like Young Diagrams

We will use conventional Young diagrams to label representations of quarks:

In [4]:
yd1 = YoungDiagram((3,2,1))

In [5]:
yd1

(3, 2, 1)

We can access the tuple corresponding to the partition:

In [6]:
yd1.partition

(3, 2, 1)

the lowest Nc for which this digram could exist:

In [7]:
yd1.N0

3

its representation for a given Nc:

In [8]:
Nc = 3
yd1_Nc3 = yd1.evaluate_for_Nc(Nc)

In [9]:
yd1_Nc3

(2, 1)

its dimension for a given Nc:

In [10]:
yd1.dimension_Nc(Nc)

8

### Antiquark-like Young Diagrams

We will use "barred" Young diagrams to label representations of quarks, arising from the adjoint representation:

In [11]:
yd2 = YoungDiagram((3,2,1),barred = True)

they are not the same objects!

In [12]:
yd1==yd2

False

But when we evaluate a barred diagram for a given Nc, it becomes a conventional, unbarred Young diagram:

In [13]:
yd2_Nc3 = yd2.evaluate_for_Nc(Nc)

In [14]:
yd2_Nc3

(2, 1)

in this case, its the same as the previous diagram when Nc=3:

In [15]:
yd2_Nc3==yd1_Nc3

True

# Multiplying Young diagrams - the DirectSum class

### (Un)barred diagram-(un)barred diagram multiplication

Uses Littlewood-Richardson rule for diagram multiplication

In [16]:
yd_unbarred = YoungDiagram((2,1))

In [17]:
ydubr_tensor_ydubr = yd_unbarred*yd_unbarred

The resulting object is a DirectSum class, which displays the direct (tensor) sum of YoungDiagram objects. The DirectSum class is a type of python dict, with keys being the YoungDiagram/Pair (more on these below) objects, and the values corresponding to the multiplicities. In the display, the constants are the multiplicities and their subscripts are the first Nc where this pair labels a young diagram.

In [18]:
ydubr_tensor_ydubr

{(2, 2, 1, 1): 1,
 (4, 1, 1): 1,
 (3, 3): 1,
 (2, 2, 2): 1,
 (3, 1, 1, 1): 1,
 (3, 2, 1): 2,
 (4, 2): 1}

the elements can be accessed using:

In [19]:
ydubr_tensor_ydubr.keys() # this returns dict_keys

dict_keys([(2, 2, 1, 1), (4, 1, 1), (3, 3), (2, 2, 2), (3, 1, 1, 1), (3, 2, 1), (4, 2)])

or:

In [20]:
ydubr_tensor_ydubr.elements()# this gives a list

[(2, 2, 1, 1), (4, 1, 1), (3, 3), (2, 2, 2), (3, 1, 1, 1), (3, 2, 1), (4, 2)]

the multiplicities can be recovered as a list in two ways as well:

In [21]:
ydubr_tensor_ydubr.values()

dict_values([1, 1, 1, 1, 1, 2, 1])

or:

In [22]:
ydubr_tensor_ydubr.multiplicities()

[1, 1, 1, 1, 1, 2, 1]

the lowest nc for each diagram can be separately recovered:

In [23]:
ydubr_tensor_ydubr.lowest_Nc()

[4, 3, 2, 3, 4, 3, 2]

We can evaluate it under a given Nc:

In [24]:
Nc = 3
ydubr_tensor_ydubr_nc3 = ydubr_tensor_ydubr.evaluate_for_Nc(Nc=Nc)

In [25]:
ydubr_tensor_ydubr_nc3

{(3): 1, (3, 3): 1, (): 1, (4, 2): 1, (2, 1): 2}

we can get the dimension in the same way:

In [26]:
ydubr_tensor_ydubr_nc3.dimension_Nc() # here the direct sum already knows which Nc we used

{8: 2, 1: 1, 10: 2, 27: 1}

and the sum:

In [27]:
ydubr_tensor_ydubr_nc3.dimension_Nc().sum()

64

Similar when multiplying two barred young diagrams:

In [28]:
yd_barred = YoungDiagram((2,1),barred = True)
ydbr_tensor_ydbr = yd_barred*yd_barred

In [29]:
ydbr_tensor_ydbr

{(4, 1, 1)_: 1,
 (3, 3)_: 1,
 (3, 1, 1, 1)_: 1,
 (2, 2, 1, 1)_: 1,
 (2, 2, 2)_: 1,
 (3, 2, 1)_: 2,
 (4, 2)_: 1}

We can also evaluate it under an Nc:

In [30]:
ydbr_tensor_ydbr_nc3 = ydbr_tensor_ydbr.evaluate_for_Nc(Nc=Nc)

In [31]:
ydbr_tensor_ydbr_nc3

{(3): 1, (3, 3): 1, (): 1, (4, 2): 1, (2, 1): 2}

this is now the same as the DirectSum containing the first tensor multiple above:

In [32]:
ydbr_tensor_ydbr_nc3==ydubr_tensor_ydubr_nc3

True

we can also get the dimensions directly (this time specifying the Nc)

In [33]:
ydbr_tensor_ydbr.dimension_Nc(Nc=Nc)

/home/anduril/Documents/birdtracks/pair_multiplication/pair_multiplication/classes.py:306: UserWarning: Young diagram pair not admissible under given Nc. Returning null diagram.
  warnings.warn('Young diagram pair not admissible under given Nc. Returning null diagram.')


{8: 2, 1: 1, 10: 2, 27: 1}

# Barred diagram-unbarred diagram multiplication - the Pair class

We do the multiplication using King's Q rule for diagram multiplication

In [34]:
yd_barred = YoungDiagram((2,1),barred = True)
yd_unbarred = YoungDiagram((2,1))

barred_tensor_unbarred = yd_barred*yd_unbarred

The results is a DirectSum of Pair objects, where the first partition is always the barred diagram, the second is always the unbarred diagram.

In [35]:
barred_tensor_unbarred

{[4]((2, 1),(2, 1)): 1,
 [4]((1, 1),(1, 1)): 1,
 [2]((2),(2)): 1,
 [3]((2),(1, 1)): 1,
 [3]((1, 1),(2)): 1,
 [2]((1),(1)): 2,
 [0]((),()): 1}

To construct a pair we can either give a tuple of partitions (the first one is always the barred one)

In [36]:
pair_from_partitions = Pair(((2,1),(2,1)))

/home/anduril/Documents/birdtracks/pair_multiplication/pair_multiplication/classes.py:504: UserWarning: Class Pair under construction. Proceed with caution!
  warnings.warn('Class Pair under construction. Proceed with caution!')


The multiple of 1 next to it stores the lowest Nc as its subscript

In [37]:
pair_from_partitions

[4]((2, 1),(2, 1))

we can also construct it using two Young diagrams:

In [38]:
yd_barred = YoungDiagram((2,1),barred = True)
yd_unbarred = YoungDiagram((2,1))

pair_from_diagrams = Pair((yd_barred,yd_unbarred))

/home/anduril/Documents/birdtracks/pair_multiplication/pair_multiplication/classes.py:504: UserWarning: Class Pair under construction. Proceed with caution!
  warnings.warn('Class Pair under construction. Proceed with caution!')


they're the same:

In [39]:
pair_from_partitions==pair_from_diagrams

True

another way is to pair one Young diagram with either a partition:

In [40]:
pair_from_diag_and_partition = yd_barred.pair_with((2,1))

/home/anduril/Documents/birdtracks/pair_multiplication/pair_multiplication/classes.py:504: UserWarning: Class Pair under construction. Proceed with caution!
  warnings.warn('Class Pair under construction. Proceed with caution!')


(when using this method, the given partition will create a diagram that is unbarred if yd_barred and vice-versa.)

We can pair a diagram with another diagram:

In [41]:
pair_from_diag_and_diag = yd_barred.pair_with(yd_unbarred)

(in this case, one must be barred and one must be unbarred, but they can be given in either order.)

they're all the same:

In [42]:
pair_from_partitions==pair_from_diag_and_diag and pair_from_diag_and_diag==pair_from_diag_and_partition

True

we can evaluate the Young diagram resulting from a given Nc in the usual way:

In [43]:
yd_Nc7 = pair_from_partitions.evaluate_for_Nc(Nc=7)

In [44]:
yd_Nc7

(4, 3, 2, 2, 2, 1)

For an Nc lower than this diagrams lowest Nc, we get a NullDiagram 

In [45]:
pair_from_partitions.evaluate_for_Nc(Nc=3)

/home/anduril/Documents/birdtracks/pair_multiplication/pair_multiplication/classes.py:625: UserWarning: Conjugate not admissible under given Nc.
  warnings.warn('Conjugate not admissible under given Nc.')


None

## Coming soon:

 - pair multiplication with diagram (in progress)
 - pair-pair multiplication (when I figure out how)
 - better handling of diagram multiplicities
 - better documentation and testing
 - more Latexing functions!